Import Pandas package so we can use Pandas dataframes and json package so we can use json files

In [67]:
import pandas as pd
import json

Function to load the .csv files containing the static tables. This function takes a list as input where each list item is the string of a pathway to a .csv file. The files are then opened as Pandas dataframes and stored in a dictionary where the filename is the key and the dataframe is the value.

In [68]:
def Load_csvs(list):
    dic_of_dfs = {}
    for i in list:
        dfname = i.rsplit('\\', 1)[-1]
        dic_of_dfs[dfname] = pd.read_csv(i)
    return dic_of_dfs

In [69]:
path_to_gest = ['C:\\Users\\josep\\OneDrive\\Documents\\GitHub\\multipeattools\\SET_Tool\\SET_Tool\\csv_files\\GEST_2_Static_Values.csv']

Function to create a json containing the user input values. The input values in this function are merely standins until actual user input is provided, at which point the json file will be updated.

In [70]:
def Create_Input_Json():
    inputs = {'gen_site_data': {'site_name': 'Dutch Example',
                                     'tot_area': 1, 
                                     'coords': [0, 0], 
                                     'elevation': 0, 
                                     'peat_type': 'Sphagnum',
                                     'peat_thick': 100,
                                     'year_start': 2022}, 
            'base': {'med_gw_level_summer': -10,
                         'veg_class': 'G5s:Wet Grassland with Shunt Species',
                         'amount_manure': 150,
                         'amount_org_fert': 10,
                         'type_synth_fert': 'Nitrate Based',
                         'amount_synth_fert': 20,
                         'type_animals': 'No Animals',
                         'avg_num_animals': 0,
                         'avg_num_days': 0,
                         'crop_yield': 12,
                         'crop_resid': 'No',
                         'crop_name': 'Cattail (Typha Sp.)',
                         'diesel_per_site': 0,
                         'elec_per_site': 0},
            'rewet': {'med_gw_level_summer': -55,
                         'veg_class': 'G2:Moist Grassland',
                         'amount_manure': 100,
                         'amount_org_fert': 50,
                         'type_synth_fert': 'Nitrate Based',
                         'amount_synth_fert': 50,
                         'type_animals': 'Sheep',
                         'avg_num_animals': 10,
                         'avg_num_days': 100,
                         'crop_yield': 24,
                         'crop_resid': 'Yes',
                         'crop_name': 'Alder (Alnus Sp.)',
                         'diesel_per_site': 10,
                         'elec_per_site': 10,
                         'crop_use': 'Paper'}}
    
    with open('user_input_SET.json', 'w') as outfile:
        print(json.dumps(inputs, indent = 5), file = outfile)

In [71]:
Create_Input_Json()

Create the Data tab by calling information from the Input json and GEST 2.0 dataframe. This function takes two parameters. The first is a string of the name of the json file where the user input data is stored, the second is the dataframe which contains the GEST 2.0 table. This function returns a pandas dataframe which contains the output provided in the data tab of the excel file.

In [72]:
def Create_Data_Tab(user_input, gest):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initiate the Data dict
    data = {'base': {}, 'rewet': {}}

    #Start populating the data dict with simple data
    data['base']['veg_class'] = user_input['base']['veg_class']
    data['base']['type_animals'] = user_input['base']['type_animals']
    data['base']['type_synth_fert'] = user_input['base']['type_synth_fert']

    data['rewet']['veg_class'] = user_input['rewet']['veg_class']
    data['rewet']['type_animals'] = user_input['rewet']['type_animals']
    data['rewet']['type_synth_fert'] = user_input['rewet']['type_synth_fert']

    #Calculate the Animal Emission Factor and Nitrogen Excretion per head per year
    if data['base']['type_animals'] == 'No Animals':
        data['base']['animal_ef'] = 0
        data['base']['n_excretion'] = 0
    elif data['base']['type_animals'] == 'Dairy Cattle':
        data['base']['animal_ef'] = 0.02
        data['base']['n_excretion'] = 130.1
    elif data['base']['type_animals'] == 'Beef Cattle':
        data['base']['animal_ef'] = 0.02
        data['base']['n_excretion'] = 28.2
    elif data['base']['type_animals'] == 'Sheep':
        data['base']['animal_ef'] = 0.01
        data['base']['n_excretion'] = 9.9
    elif data['base']['type_animals'] == 'Goats':
        data['base']['animal_ef'] = 0.01
        data['base']['n_excretion'] = 7.4
    elif data['base']['type_animals'] == 'Water Buffalo':
        data['base']['animal_ef'] = 0.02
        data['base']['n_excretion'] = 76.5

    if data['rewet']['type_animals'] == data['base']['type_animals']:
        data['rewet']['animal_ef'] = data['base']['animal_ef']
        data['rewet']['n_excretion'] = data['rewet']['n_excretion']
    else:
        data['rewet']['animal_ef'] = 0
        data['rewet']['n_excretion'] = 0

    #Calculate Synthetic Fertilizer emission factor
    if data['base']['type_synth_fert'] == 'Nitrate Based':
        data['base']['fert_ef'] = 0.015
    elif data['base']['type_synth_fert'] == 'Ammonium Based':
        data['base']['fert_ef'] = 0.01

    if data['rewet']['type_synth_fert'] == 'Nitrate Based':
        data['rewet']['fert_ef'] = 0.015
    elif data['rewet']['type_synth_fert'] == 'Ammonium Based':
        data['rewet']['fert_ef'] = 0.01

    #Find Crop Residues number
    if user_input['base']['crop_resid'] == 'Yes':
        data['base']['crop_resid'] = 1
    else:
        data['base']['crop_resid'] = 2

    if user_input['rewet']['crop_resid'] == 'Yes':
        data['rewet']['crop_resid'] = 1
    else:
        data['rewet']['crop_resid'] = 2

    #Find Number of crop Name
    if user_input['base']['crop_name'] == 'Cattail (Typha Sp.)':
        data['base']['crop_name'] = 1
    elif user_input['base']['crop_name'] == 'Reed (Phragmites Australis)':
        data['base']['crop_name'] = 2
    elif user_input['base']['crop_name'] == 'Peat Moss (Sphagnum Sp.) ':
        data['base']['crop_name'] = 3
    elif user_input['base']['crop_name'] == 'Grasses like Reed Canary Grass':
        data['base']['crop_name'] = 4
    elif user_input['base']['crop_name'] == 'Alder (Alnus Sp.)':
        data['base']['crop_name'] = 5
    elif user_input['base']['crop_name'] == 'Other':
        data['base']['crop_name'] = 6

    if user_input['rewet']['crop_name'] == 'Cattail (Typha Sp.)':
        data['rewet']['crop_name'] = 1
    elif user_input['rewet']['crop_name'] == 'Reed (Phragmites Australis)':
        data['rewet']['crop_name'] = 2
    elif user_input['rewet']['crop_name'] == 'Peat Moss (Sphagnum Sp.) ':
        data['rewet']['crop_name'] = 3
    elif user_input['rewet']['crop_name'] == 'Grasses like Reed Canary Grass':
        data['rewet']['crop_name'] = 4
    elif user_input['rewet']['crop_name'] == 'Alder (Alnus Sp.)':
        data['rewet']['crop_name'] = 5
    elif user_input['rewet']['crop_name'] == 'Other':
        data['rewet']['crop_name'] = 6

    #Find Product Use Number
    if user_input['rewet']['crop_use'] == 'Building Materials e.g insulation, taching, timber':
        data['rewet']['crop_use'] = 1
    elif user_input['rewet']['crop_use'] == 'Bedding Material':
        data['rewet']['crop_use'] = 2
    elif user_input['rewet']['crop_use'] == 'Food Application':
        data['rewet']['crop_use'] = 3
    elif user_input['rewet']['crop_use'] == 'Fodder/Feed Application':
        data['rewet']['crop_use'] = 4
    elif user_input['rewet']['crop_use'] == 'Energy Use: biogas, combustion, wood etc':
        data['rewet']['crop_use'] = 5
    elif user_input['rewet']['crop_use'] == 'Paper':
        data['rewet']['crop_use'] = 6
    elif user_input['rewet']['crop_use'] == 'Extraction of Ingredients/Building Blocks: proteins, fibres, cellulose etc':
        data['rewet']['crop_use'] = 7
    elif user_input['rewet']['crop_use'] == 'High Quality Substrate in Horticulture':
        data['rewet']['crop_use'] = 8
    elif user_input['rewet']['crop_use'] == 'Other Uses/Unknown':
        data['rewet']['crop_use'] = 9

    #Perform Vegetation Class emission calculations
    veg_num_base = data['base']['veg_class'].split(':', 1)[0]
    veg_num_rewet = data['rewet']['veg_class'].split(':', 1)[0]
    
    for i in range(2, 44):
        if veg_num_base == gest.iloc[i]['Name']:
            data['base']['veg_ch4_gwp'] = gest.iloc[i]['CH4']
            data['base']['veg_c02_gwp'] = gest.iloc[i]['CO2']
            data['base']['veg_cflux_gwp'] = gest.iloc[i]['Total C-flux (GWP)']
        if veg_num_rewet == gest.iloc[i]['Name']:
            data['rewet']['veg_ch4_gwp'] = gest.iloc[i]['CH4']
            data['rewet']['veg_c02_gwp'] = gest.iloc[i]['CO2']
            data['rewet']['veg_cflux_gwp'] = gest.iloc[i]['Total C-flux (GWP)']

    return pd.DataFrame.from_dict(data)
    


In [73]:
dict_of_csvs = Load_csvs(path_to_gest)
gest = dict_of_csvs['GEST_2_Static_Values.csv']

In [74]:
data_tab = Create_Data_Tab('user_input_SET.json', gest)
data_tab

,base,rewet
veg_class,G5s:Wet Grassland with Shunt Species,G2:Moist Grassland
type_animals,No Animals,Sheep
type_synth_fert,Nitrate Based,Nitrate Based
animal_ef,0,0
n_excretion,0,0
fert_ef,0.015,0.015
crop_resid,2,1
crop_name,1,5
veg_ch4_gwp,2.93,0.01
veg_c02_gwp,-3.89,19.37


Create the Data tab by calling information from the Input json and the Data Tab dataframe. This function takes two parameters. The first is a string of the name of the json file where the user input data is stored, the second is the dataframe which contains the Data Tab dataframe. This function returns a pandas dataframe which contains the output provided in the data tab of the excel file.

In [75]:
def Create_crop_Use_Tab(user_input, data):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize crop_use dictionary
    crop_use = {}

    #Populate the crop_use dictionary
    crop_use['crop_yield'] = user_input['rewet']['crop_yield']

    if data.loc['crop_use']['rewet'] == 2 or data.loc['crop_use']['rewet'] == 3 or data.loc['crop_use']['rewet'] == 4 or data.loc['crop_use']['rewet'] == 9:
        crop_use['product_weight'] = 0
    else:
        crop_use['product_weight'] = crop_use['crop_yield']

    crop_use['carbon_weight'] = crop_use['product_weight']*0.475
    crop_use['ton_co2'] = crop_use['carbon_weight']*(44/12)
    crop_use['ton_co2_per_ha'] = crop_use['ton_co2']*(-1)
    crop_use['ton_co2_per_site'] = crop_use['ton_co2_per_ha']*user_input['gen_site_data']['tot_area']

    #Save crop_use as a pandas dataframe
    crop_use = {'Values': crop_use}
    return pd.DataFrame.from_dict(crop_use)

In [76]:
crop_use_tab = Create_crop_Use_Tab('user_input_SET.json', data_tab)
crop_use_tab

,Values
carbon_weight,11.4
crop_yield,24.0
product_weight,24.0
ton_co2,41.8
ton_co2_per_ha,-41.8
ton_co2_per_site,-41.8


Create the C Content Soil Tab by using information from the User Input json.

In [77]:
def Create_C_Content_Soil_Tab(user_input):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize the C Content Soil dictionary
    c_content = {}

    #Populate C Content dictionary
    c_content['peat_type'] = user_input['gen_site_data']['peat_type']

    if c_content['peat_type'] == 'Sphagnum':
        c_content['c_content_per_cm_thick'] = 0.3496
    elif c_content['peat_type'] == 'Herbaceous':
        c_content['c_content_per_cm_thick'] = 0.5959
    elif c_content['peat_type'] == 'Woody':
        c_content['c_content_per_cm_thick'] = 0.54972
    elif c_content['peat_type'] == 'Brown moss':
        c_content['c_content_per_cm_thick'] = 0.84783
    elif c_content['peat_type'] == 'Unknown':
        c_content['c_content_per_cm_thick'] = 0.55224
    elif c_content['peat_type'] == 'Humified':
        c_content['c_content_per_cm_thick'] = 0.91008

    c_content['peat_thick'] = user_input['gen_site_data']['peat_thick']
    c_content['c_stock_ton_per_ha'] = c_content['peat_thick']*c_content['c_content_per_cm_thick']*10
    c_content['c_stock_tco2_per_ha'] = c_content['c_stock_ton_per_ha']*(44/12)

    #Save C Content Soil tab as a pandas dataframe
    c_content = {'Values': c_content}
    return pd.DataFrame.from_dict(c_content)

In [78]:
c_content_tab = Create_C_Content_Soil_Tab('user_input_SET.json')
c_content_tab

,Values
c_content_per_cm_thick,0.3496
c_stock_tco2_per_ha,1281.866667
c_stock_ton_per_ha,349.6
peat_thick,100
peat_type,Sphagnum


Create the Soil Moisture Classes Tab by using information from the User Input json.

In [79]:
def Create_Soil_Moisture_Classes_Tab(user_input):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize the Soil Moisture Classes dictionary
    sm_classes = {'base': {}, 'rewet': {}}

    #Populate dictionary
    if user_input['base']['med_gw_level_summer'] >= 0 and user_input['base']['med_gw_level_summer'] <= 140:
        sm_classes['base']['summer_moist_class_num'] = '6+'
        sm_classes['base']['summer_moist_class_name'] = 'Flooded (lower eulittoral)'
        sm_classes['base']['summer_moist_class'] = '6+ (Flooded (lower eulittoral))'
    elif user_input['base']['med_gw_level_summer'] >= -10 and user_input['base']['med_gw_level_summer'] < 0:
        sm_classes['base']['summer_moist_class_num'] = '5+'
        sm_classes['base']['summer_moist_class_name'] = 'Wet (upper eulittoral)'
        sm_classes['base']['summer_moist_class'] = '5+ (Wet (upper eulittoral))'
    elif user_input['base']['med_gw_level_summer'] >= -20 and user_input['base']['med_gw_level_summer'] < -10:
        sm_classes['base']['summer_moist_class_num'] = '4+'
        sm_classes['base']['summer_moist_class_name'] = 'Semi wet (very moist)'
        sm_classes['base']['summer_moist_class'] = '4+ (Semi wet (very moist))'
    elif user_input['base']['med_gw_level_summer'] >= -45 and user_input['base']['med_gw_level_summer'] < -20:
        sm_classes['base']['summer_moist_class_num'] = '3+'
        sm_classes['base']['summer_moist_class_name'] = 'Moist'
        sm_classes['base']['summer_moist_class'] = '3+ (Moist)'
    elif user_input['base']['med_gw_level_summer'] >= -85 and user_input['base']['med_gw_level_summer'] < -45:
        sm_classes['base']['summer_moist_class_num'] = '2+'
        sm_classes['base']['summer_moist_class_name'] = 'Moderate moist'
        sm_classes['base']['summer_moist_class'] = '2+ (Moderate moist)'
    elif user_input['base']['med_gw_level_summer'] < -85:
        sm_classes['base']['summer_moist_class_num'] = '2-'
        sm_classes['base']['summer_moist_class_name'] = 'Moderate dry'
        sm_classes['base']['summer_moist_class'] = '2- (Moderate dry)'

    if user_input['rewet']['med_gw_level_summer'] >= 0 and user_input['rewet']['med_gw_level_summer'] <= 140:
        sm_classes['rewet']['summer_moist_class_num'] = '6+'
        sm_classes['rewet']['summer_moist_class_name'] = 'Flooded (lower eulittoral)'
        sm_classes['rewet']['summer_moist_class'] = '6+ (Flooded (lower eulittoral))'
    elif user_input['rewet']['med_gw_level_summer'] >= -10 and user_input['rewet']['med_gw_level_summer'] < 0:
        sm_classes['rewet']['summer_moist_class_num'] = '5+'
        sm_classes['rewet']['summer_moist_class_name'] = 'Wet (upper eulittoral)'
        sm_classes['rewet']['summer_moist_class'] = '5+ (Wet (upper eulittoral))'
    elif user_input['rewet']['med_gw_level_summer'] >= -20 and user_input['rewet']['med_gw_level_summer'] < -10:
        sm_classes['rewet']['summer_moist_class_num'] = '4+'
        sm_classes['rewet']['summer_moist_class_name'] = 'Semi wet (very moist)'
        sm_classes['rewet']['summer_moist_class'] = '4+ (Semi wet (very moist))'
    elif user_input['rewet']['med_gw_level_summer'] >= -45 and user_input['rewet']['med_gw_level_summer'] < -20:
        sm_classes['rewet']['summer_moist_class_num'] = '3+'
        sm_classes['rewet']['summer_moist_class_name'] = 'Moist'
        sm_classes['rewet']['summer_moist_class'] = '3+ (Moist)'
    elif user_input['rewet']['med_gw_level_summer'] >= -85 and user_input['rewet']['med_gw_level_summer'] < -45:
        sm_classes['rewet']['summer_moist_class_num'] = '2+'
        sm_classes['rewet']['summer_moist_class_name'] = 'Moderate moist'
        sm_classes['rewet']['summer_moist_class'] = '2+ (Moderate moist)'
    elif user_input['rewet']['med_gw_level_summer'] < -85:
        sm_classes['rewet']['summer_moist_class_num'] = '2-'
        sm_classes['rewet']['summer_moist_class_name'] = 'Moderate dry'
        sm_classes['rewet']['summer_moist_class'] = '2- (Moderate dry)'

    #Save Soil Moisture Classes tab as a pandas dataframe
    return pd.DataFrame.from_dict(sm_classes)

In [80]:
sm_classes = Create_Soil_Moisture_Classes_Tab('user_input_SET.json')
sm_classes

,base,rewet
summer_moist_class_num,5+,2+
summer_moist_class_name,Wet (upper eulittoral),Moderate moist
summer_moist_class,5+ (Wet (upper eulittoral)),2+ (Moderate moist)


Functions corresponding to calculations used in the N2O fertilizer tab

In [81]:
#Direct N2O emissions

def Manure_CO2Site_Base(manure_applied_base, tot_area):
    EF=0.02
    N2O_CO2eq=265 #value from the CO2 equivalent table, just this value is used from the table
    N2O_N=EF*manure_applied_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_manure_base=N2O_site*N2O_CO2eq
    
    return CO2_site_manure_base

def Organic_Fert_CO2Site_Base(organic_applied_base,tot_area):
    EF=0.02
    N2O_CO2eq=265 
    N2O_N=EF*organic_applied_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_organic_base=N2O_site*N2O_CO2eq
    
    return CO2_site_organic_base

def Grazing_CO2Site_Base(avg_n_animals_base, avg_days_base, n_excretion_value_b, EF_animal_b, tot_area):
    N2O_CO2eq=265
    N_amount_base=(n_excretion_value_b*avg_n_animals_base*(avg_days_base/365)) / tot_area
    N2O_N=EF_animal_b*N_amount_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_grazing_base=N2O_site*N2O_CO2eq
    
    return CO2_site_grazing_base, N_amount_base

def Synth_CO2Site_Base(fert_applied_base, EF_fert_b, tot_area): 
    N2O_CO2eq=265 
    N2O_N=EF_fert_b*fert_applied_base
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_synth_fert_base=N2O_site*N2O_CO2eq
    
    return CO2_site_synth_fert_base

def Residue_Left_Input_Base(answer_b, crop_b):
    crops = ["Cattail (Typha sp.)", "Reed (Pharagmites australis)", "Peat moss (Sphagnum sp.)", "Grasses like reed canary grass", "Alder (Alnus sp.)", "Other" ]  # List of all crop names
    
    if answer_b == "Yes" and crop_b in crops:
        
        if crop_b == "Cattail (Typha sp.)":
            cropresidue_fraction_tot_yield_b = 0.11627907
        elif crop_b == "Reed (Pharagmites australis)":
            cropresidue_fraction_tot_yield_b = 0.046511628
        else:
            cropresidue_fraction_tot_yield_b = 0.0
            
    else:
        cropresidue_fraction_tot_yield_b = 0.0
    
    return cropresidue_fraction_tot_yield_b

def crop_Residue_Base(cropresidue_fraction_tot_yield_b, crop_yield_base, tot_area):
    nitrogen_content=0.015
    EF=0.02
    N2O_CO2eq=265
    amount_applied=cropresidue_fraction_tot_yield_b*crop_yield_base*1000*nitrogen_content
    N2O_N=EF*amount_applied
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_cropres_base=N2O_site*N2O_CO2eq
    
    return CO2_site_cropres_base

def Basis_value_managed_soils(tot_area):
    N2O_CO2eq=265
    N2O_ha=8 #Not sure why this value is 8
    N2O_site=N2O_ha*tot_area
    CO2_site_managed_soil=N2O_site*N2O_CO2eq
    
    return  CO2_site_managed_soil

def Total_Direct_N2Oemissions_Base(CO2_site_managed_soil, CO2_site_cropres_base, CO2_site_synth_fert_base, CO2_site_grazing_base, CO2_site_organic_base, CO2_site_manure_base ):
    total_direct_N2Oemiss_base= (CO2_site_managed_soil + CO2_site_cropres_base + CO2_site_synth_fert_base + CO2_site_grazing_base + CO2_site_organic_base + CO2_site_manure_base)/1000
    
    return total_direct_N2Oemiss_base

In [82]:
#Indirect N2O Emissions

def Animal_Ammonia_Base(manure_applied_base, N_amount_base, tot_area, avg_animals, avg_days):
    EF_ammonia=0.104
    EF_N2O_N=0.01
    N2O_CO2eq=265
    ammonia_applied= manure_applied_base + ((N_amount_base*avg_animals*(avg_days/365))/tot_area)
    volatilization_N= ammonia_applied*EF_ammonia
    N2O_N=EF_N2O_N*volatilization_N
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_animal_amm_base=N2O_site*N2O_CO2eq
    
    return CO2_site_animal_amm_base

def Fert_Ammonia_Base(fert_applied_base, EF_ammonia_b, tot_area):
    EF_N2O_N=0.01
    N2O_CO2eq=265
    volatilization_N=fert_applied_base*EF_ammonia_b
    N2O_N=EF_N2O_N*volatilization_N
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_fert_amm_base=N2O_site*N2O_CO2eq
    
    return CO2_site_fert_amm_base

def N_Oxide_Base(manure_applied_base, fert_applied_base, tot_area):
    EF_NOxide_ammonia=0.15
    EF_N2O_N=0.01
    N2O_CO2eq=265
    N=manure_applied_base + fert_applied_base
    volatilization_N= EF_NOxide_ammonia*N
    N2O_N=EF_N2O_N*volatilization_N
    N2O_ha= N2O_N*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_NOxide_base=N2O_site*N2O_CO2eq
    
    return CO2_site_NOxide_base

def Nitrate_Base(manure_applied_base, organic_applied_base, cropresidue_fraction_tot_yield_b, fert_applied_base, tot_area):
    EF_nitrate_leaching=0.3
    EF_N2O_N_leached=0.025
    N2O_CO2eq=265
    applied_animal=manure_applied_base + organic_applied_base
    applied_remaining=organic_applied_base + cropresidue_fraction_tot_yield_b
    nitrate_leached= (applied_animal +  fert_applied_base +  applied_remaining) * EF_nitrate_leaching
    N2O_N_leached= nitrate_leached * EF_N2O_N_leached
    N2O_ha= N2O_N_leached*(44/28)
    N2O_site=N2O_ha*tot_area
    CO2_site_nitrate_base=N2O_site*N2O_CO2eq
    
    return CO2_site_nitrate_base

def Total_Indirect_N2Oemissions_Base( CO2_site_nitrate_base, CO2_site_NOxide_base, CO2_site_fert_amm_base, CO2_site_animal_amm_base):
    tot_indirect_N2Oemiss_base=(CO2_site_nitrate_base + CO2_site_NOxide_base + CO2_site_fert_amm_base + CO2_site_animal_amm_base)/1000
    return tot_indirect_N2Oemiss_base

Create a function for the construction of the Outcome tab using information from the user input json and the N2O, crop_use, Data, and C Content Soil tabs, as well as the GEST 2.0 csv.

In [83]:
def Create_Outcome_Tab(user_input, data, crop_use, c_content, gest):

    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialize the Soil Moisture Classes dictionary
    outcome = {'base': {}, 'rewet': {}, 'creditable_year': {}}

    #Populate the base dictionary
    outcome['base']['veg_ch4_gwp'] = data.loc['veg_ch4_gwp']['base']*user_input['gen_site_data']['tot_area']
    outcome['base']['veg_c02_gwp'] = data.loc['veg_c02_gwp']['base']*user_input['gen_site_data']['tot_area']

    outcome['base']['tot_direct_n2o'] = Total_Direct_N2Oemissions_Base(Basis_value_managed_soils(user_input['gen_site_data']['tot_area']), 
                                                                            crop_Residue_Base(Residue_Left_Input_Base(user_input['base']['crop_resid'], user_input['base']['crop_name']), user_input['base']['crop_yield'], user_input['gen_site_data']['tot_area']), 
                                                                            Synth_CO2Site_Base(user_input['base']['amount_synth_fert'], data.loc['fert_ef']['base'], user_input['gen_site_data']['tot_area']),
                                                                            Grazing_CO2Site_Base(user_input['base']['avg_num_animals'], user_input['base']['avg_num_days'], data.loc['n_excretion']['base'], data.loc['animal_ef']['base'], user_input['gen_site_data']['tot_area'])[0],
                                                                            Organic_Fert_CO2Site_Base(user_input['base']['amount_org_fert'], user_input['gen_site_data']['tot_area']),
                                                                            Manure_CO2Site_Base(user_input['base']['amount_manure'], user_input['gen_site_data']['tot_area']))

    outcome['base']['tot_indirect_n2o'] = Total_Indirect_N2Oemissions_Base(Nitrate_Base(user_input['base']['amount_manure'], user_input['base']['amount_org_fert'], Residue_Left_Input_Base(user_input['base']['crop_resid'], user_input['base']['crop_name']), user_input['base']['amount_synth_fert'], user_input['gen_site_data']['tot_area']), 
                                                                                 N_Oxide_Base(user_input['base']['amount_manure'], user_input['base']['amount_synth_fert'], user_input['gen_site_data']['tot_area']), 
                                                                                 Fert_Ammonia_Base(user_input['base']['amount_synth_fert'], data.loc['fert_ef']['base'], user_input['gen_site_data']['tot_area']), 
                                                                                 Animal_Ammonia_Base(user_input['base']['amount_manure'], data.loc['n_excretion']['base'], user_input['gen_site_data']['tot_area'], user_input['base']['avg_num_animals'], user_input['base']['avg_num_days']))
    
    outcome['base']['activity'] = ((user_input['base']['diesel_per_site']*3.35)+(user_input['base']['elec_per_site']*0.581))/1000
    outcome['base']['total'] = sum((float(outcome['base']['veg_ch4_gwp']), float(outcome['base']['veg_c02_gwp']), float(outcome['base']['tot_direct_n2o']), float(outcome['base']['tot_indirect_n2o']), float(outcome['base']['activity'])), 0)
    
    #Populate the Rewetting dictionary
    outcome['rewet']['veg_ch4_gwp'] = data.loc['veg_ch4_gwp']['rewet']*user_input['gen_site_data']['tot_area']
    outcome['rewet']['veg_c02_gwp'] = data.loc['veg_c02_gwp']['rewet']*user_input['gen_site_data']['tot_area']

    outcome['rewet']['tot_direct_n2o'] = Total_Direct_N2Oemissions_Base(Basis_value_managed_soils(user_input['gen_site_data']['tot_area']), 
                                                                            crop_Residue_Base(Residue_Left_Input_Base(user_input['rewet']['crop_resid'], user_input['rewet']['crop_name']), user_input['rewet']['crop_yield'], user_input['gen_site_data']['tot_area']), 
                                                                            Synth_CO2Site_Base(user_input['rewet']['amount_synth_fert'], data.loc['fert_ef']['rewet'], user_input['gen_site_data']['tot_area']),
                                                                            Grazing_CO2Site_Base(user_input['rewet']['avg_num_animals'], user_input['rewet']['avg_num_days'], data.loc['n_excretion']['rewet'], data.loc['animal_ef']['rewet'], user_input['gen_site_data']['tot_area'])[0],
                                                                            Organic_Fert_CO2Site_Base(user_input['rewet']['amount_org_fert'], user_input['gen_site_data']['tot_area']),
                                                                            Manure_CO2Site_Base(user_input['rewet']['amount_org_fert'], user_input['gen_site_data']['tot_area']))
    
    outcome['rewet']['tot_indirect_n2o'] = Total_Indirect_N2Oemissions_Base(Nitrate_Base(user_input['rewet']['amount_manure'], user_input['rewet']['amount_org_fert'], Residue_Left_Input_Base(user_input['rewet']['crop_resid'], user_input['rewet']['crop_name']), user_input['rewet']['amount_synth_fert'], user_input['gen_site_data']['tot_area']), 
                                                                                 N_Oxide_Base(user_input['rewet']['amount_manure'], user_input['rewet']['amount_synth_fert'], user_input['gen_site_data']['tot_area']), 
                                                                                 Fert_Ammonia_Base(user_input['rewet']['amount_synth_fert'], data.loc['fert_ef']['rewet'], user_input['gen_site_data']['tot_area']), 
                                                                                 Animal_Ammonia_Base(user_input['rewet']['amount_manure'], data.loc['n_excretion']['rewet'], user_input['gen_site_data']['tot_area'], user_input['rewet']['avg_num_animals'], user_input['rewet']['avg_num_days']))
    
    outcome['rewet']['Product ton_co2_per_site'] = crop_use.loc['ton_co2_per_site']['Values']
    outcome['rewet']['activity'] = ((user_input['rewet']['diesel_per_site']*3.35)+(user_input['rewet']['elec_per_site']*0.581))/1000
    outcome['rewet']['total'] = sum((float(outcome['rewet']['veg_ch4_gwp']), float(outcome['rewet']['veg_c02_gwp']), float(outcome['rewet']['tot_direct_n2o']), float(outcome['rewet']['tot_indirect_n2o']), float(outcome['rewet']['activity']), float(outcome['rewet']['Product ton_co2_per_site'])), 0)

    #Populate creditable_year tab
    veg_num_base = data['base']['veg_class'].split(':', 1)[0]
    veg_num_rewet = data['rewet']['veg_class'].split(':', 1)[0]
    
    for i in range(2, 44):
        if veg_num_base == gest.iloc[i]['Name']:
            outcome['creditable_year']['base_scenario'] = float(c_content.loc['c_stock_ton_per_ha']['Values'])/(float(gest.iloc[i]['Total C-flux (ton C/ha)']))
        
        if veg_num_rewet == gest.iloc[i]['Name']:
            if float(c_content.loc['c_stock_ton_per_ha']['Values'])/float(gest.iloc[i]['Total C-flux (ton C/ha)']) > 0:
                outcome['creditable_year']['rewet_scenario'] = float(c_content.loc['c_stock_ton_per_ha']['Values'])/float(gest.iloc[i]['Total C-flux (ton C/ha)'])
            else:
                outcome['creditable_year']['rewet_scenario'] = float('inf')
    
    #Save the outcome tab in a pandas database
    return pd.DataFrame.from_dict(outcome)


In [84]:
outcome = Create_Outcome_Tab('user_input_SET.json', data_tab, crop_use_tab, c_content_tab, gest)
outcome

,base,rewet,creditable_year
veg_ch4_gwp,2.93,0.01,NaN
veg_c02_gwp,-3.89,19.37,NaN
tot_direct_n2o,3.5775,3.265179,NaN
tot_indirect_n2o,0.765812,0.920932,NaN
activity,0.0,0.03931,NaN
total,3.383312,-18.19458,NaN
Product ton_co2_per_site,NaN,-41.8,NaN
rewet_scenario,NaN,NaN,66.174180
base_scenario,NaN,NaN,-359.297774


Create Timeline tab using information from the user input json, GEST 2.0 csv, and the outcome and C content soil tabs.

In [85]:
def Create_Timeline_tab(user_input, outcome, c_content, gest):
        
    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)
        
    #Initialize the Timeline dict
    timeline = {}
    list_colnames = ['base_emissions', 'base_GESTv2', 'rewet_emissions', 'rewet_GESTv2', 'carbon_savings_flow', 'carbon_savings_stock', 'carbon_savings_product', 'carbon_savings_total', 'base_GESTnr', 'c_sequestration_base', 'c_stock_soil_base', 'stock_savings_creditable', 'rewet_GESTnr', 'c_sequestration_rewet', 'c_credits_' + user_input['gen_site_data']['site_name']]
    for i in list_colnames:
        timeline[i] = {}
        for j in range(int(user_input['gen_site_data']['year_start']), int(user_input['gen_site_data']['year_start'])+50):
            timeline[i][j] = None

    #Populate dict column by column
    for i in range(list(timeline['base_emissions'].keys())[0], list(timeline['base_emissions'].keys())[-1]+1):
        timeline['base_emissions'][i] = outcome.loc['total']['base']
        timeline['base_GESTv2'][i] = user_input['base']['veg_class']
        timeline['rewet_emissions'][i] = outcome.loc['total']['rewet']
        timeline['rewet_GESTv2'][i] = user_input['rewet']['veg_class']
        timeline['carbon_savings_flow'][i] = float(outcome.loc['tot_direct_n2o']['base']) + float(outcome.loc['tot_indirect_n2o']['base']) + float(outcome.loc['activity']['base']) - float(outcome.loc['tot_direct_n2o']['rewet']) - float(outcome.loc['tot_indirect_n2o']['rewet']) - float(outcome.loc['activity']['rewet'])
        timeline['carbon_savings_stock'][i] = float(outcome.loc['veg_ch4_gwp']['base']) + float(outcome.loc['veg_c02_gwp']['base']) - float(outcome.loc['veg_ch4_gwp']['rewet']) - float(outcome.loc['veg_c02_gwp']['rewet'])
        timeline['carbon_savings_product'][i] = (-1)*float(outcome.loc['Product ton_co2_per_site']['rewet'])
        timeline['carbon_savings_total'][i] = sum((timeline['carbon_savings_flow'][i], timeline['carbon_savings_stock'][i], timeline['carbon_savings_product'][i]), 0)
        timeline['base_GESTnr'][i] = timeline['base_GESTv2'][i].split(':', 1)[0]
        timeline['rewet_GESTnr'][i] = timeline['rewet_GESTv2'][i].split(':', 1)[0]

        for j in range(2, 44):
            if timeline['base_GESTnr'][i] == gest.iloc[j]['Name']:
                timeline['c_sequestration_base'][i] = float(gest.iloc[j]['Total C-flux (ton C/ha)'])*user_input['gen_site_data']['tot_area']*(-1)
            if timeline['rewet_GESTnr'][i] == gest.iloc[j]['Name']:
                timeline['c_sequestration_rewet'][i] = float(gest.iloc[j]['Total C-flux (ton C/ha)'])*user_input['gen_site_data']['tot_area']*(-1)

        if i == list(timeline['base_emissions'].keys())[0]:
            timeline['c_stock_soil_base'][i] = (c_content.loc['c_stock_ton_per_ha']['Values']*user_input['gen_site_data']['tot_area']) + timeline['c_sequestration_base'][i]
        else:
            timeline['c_stock_soil_base'][i] = timeline['c_stock_soil_base'][i-1] + timeline['c_sequestration_base'][i]

        if timeline['c_stock_soil_base'][i] > 0:
            timeline['stock_savings_creditable'][i] = 1
        else:
            timeline['stock_savings_creditable'][i] = 0

        if timeline['stock_savings_creditable'][i] == 1:
            timeline['c_credits_' + user_input['gen_site_data']['site_name']][i] = timeline['carbon_savings_total'][i]
        else:
            if timeline['c_sequestration_rewet'][i] > 0:
                timeline['c_credits_' + user_input['gen_site_data']['site_name']][i] = timeline['carbon_savings_flow'][i] + timeline['carbon_savings_product'][i] + (timeline['c_sequestration_rewet'][i]*(44/12))
            else:
                timeline['c_credits_' + user_input['gen_site_data']['site_name']][i] = timeline['carbon_savings_flow'][i] + timeline['carbon_savings_product'][i]
            
        

    #Save the outcome tab in a pandas database
    return pd.DataFrame.from_dict(timeline)
    

In [89]:
timeline = Create_Timeline_tab('user_input_SET.json', outcome, c_content_tab, gest)
timeline

,base_emissions,base_GESTv2,rewet_emissions,rewet_GESTv2,carbon_savings_flow,carbon_savings_stock,carbon_savings_product,carbon_savings_total,base_GESTnr,c_sequestration_base,c_stock_soil_base,stock_savings_creditable,rewet_GESTnr,c_sequestration_rewet,c_credits_Dutch Example
2022,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,350.573009,1,G2,-5.283027,21.577892
2023,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,351.546018,1,G2,-5.283027,21.577892
2024,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,352.519027,1,G2,-5.283027,21.577892
2025,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,353.492036,1,G2,-5.283027,21.577892
2026,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,354.465045,1,G2,-5.283027,21.577892
2027,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,355.438055,1,G2,-5.283027,21.577892
2028,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,356.411064,1,G2,-5.283027,21.577892
2029,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,357.384073,1,G2,-5.283027,21.577892
2030,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,358.357082,1,G2,-5.283027,21.577892
2031,3.383312,G5s:Wet Grassland with Shunt Species,-18.19458,G2:Moist Grassland,0.117892,-20.34,41.8,21.577892,G5s,0.973009,359.330091,1,G2,-5.283027,21.577892


Create Output tab using information from the user input json, soil moisture classes, data, outcome, and C content soil tabs.

In [92]:
def Create_Output_tab(user_input, sm_classes, data_tab, outcome, c_content, crop_use):
    
    #Open the json file and convert to dict
    with open(user_input) as json_file:
        user_input = json.load(json_file)

    #Initialise Output dict
    output = {'site_data': {}, 'base_outcomes': {}, 'rewet_outcomes': {}, 'carbon_savings': {}}

    #Populate the site_data section
    output['site_data']['site_name'] = user_input['gen_site_data']['site_name']
    output['site_data']['coords'] = user_input['gen_site_data']['coords']
    output['site_data']['tot_area'] = user_input['gen_site_data']['tot_area']
    output['site_data']['peat_type'] = user_input['gen_site_data']['peat_type']
    output['site_data']['peat_thick'] = user_input['gen_site_data']['peat_thick']
    output['site_data']['year_start'] = user_input['gen_site_data']['year_start']
    output['site_data']['sm_class_base'] = sm_classes.loc['summer_moist_class']['base']
    output['site_data']['sm_class_rewet'] = sm_classes.loc['summer_moist_class']['rewet']
    output['site_data']['base veg_class'] = user_input['base']['veg_class']
    output['site_data']['rewet_veg_class'] = user_input['rewet']['veg_class']

    #Populate the base_outcomes section
    output['base_outcomes']['CH4'] = float(data_tab.loc['veg_ch4_gwp']['base']*user_input['gen_site_data']['tot_area'])
    output['base_outcomes']['CO2'] = float(data_tab.loc['veg_c02_gwp']['base']*user_input['gen_site_data']['tot_area'])
    output['base_outcomes']['c_emission_gwp_subtotal'] = float(output['base_outcomes']['CH4']) + float(output['base_outcomes']['CO2'])

    output['base_outcomes']['n2o_direct'] = outcome.loc['tot_direct_n2o']['base']
    output['base_outcomes']['n2o_indirect'] = outcome.loc['tot_indirect_n2o']['base']
    output['base_outcomes']['n2o_emission_gwp_subtotal'] = float(output['base_outcomes']['n2o_direct']) + float(output['base_outcomes']['n2o_indirect'])

    output['base_outcomes']['activity_gwp_subtotal'] = outcome.loc['activity']['base']
    output['base_outcomes']['gwp_total'] = outcome.loc['total']['base']

    #Populate the rewet_outcomes section
    output['rewet_outcomes']['CH4'] = float(data_tab.loc['veg_ch4_gwp']['rewet']*user_input['gen_site_data']['tot_area'])
    output['rewet_outcomes']['CO2'] = float(data_tab.loc['veg_c02_gwp']['rewet']*user_input['gen_site_data']['tot_area'])
    output['rewet_outcomes']['c_emission_gwp_subtotal'] = float(output['rewet_outcomes']['CH4']) + float(output['rewet_outcomes']['CO2'])

    output['rewet_outcomes']['n2o_direct'] = outcome.loc['tot_direct_n2o']['rewet']
    output['rewet_outcomes']['n2o_indirect'] = outcome.loc['tot_indirect_n2o']['rewet']
    output['rewet_outcomes']['n2o_emission_gwp_subtotal'] = float(output['rewet_outcomes']['n2o_direct']) + float(output['rewet_outcomes']['n2o_indirect'])

    output['rewet_outcomes']['activity_gwp_subtotal'] = outcome.loc['activity']['rewet']
    output['rewet_outcomes']['product_gwp_subtotal'] = outcome.loc['Product ton_co2_per_site']['rewet']
    output['rewet_outcomes']['gwp_total'] = outcome.loc['total']['rewet']

    #Populate carbon_savings section
    output['carbon_savings']['ghg_savings_total_per_year_per_site'] = output['base_outcomes']['gwp_total'] - output['rewet_outcomes']['gwp_total']
    output['carbon_savings']['ghg_savings_total_per_year_per_ha'] = output['carbon_savings']['ghg_savings_total_per_year_per_site']/output['site_data']['tot_area']
    output['carbon_savings']['ghg_savings_stock_per_year_per_site'] = float(outcome.loc['veg_ch4_gwp']['base']) + float(outcome.loc['veg_c02_gwp']['base']) - float(outcome.loc['veg_ch4_gwp']['rewet']) - float(outcome.loc['veg_c02_gwp']['rewet'])
    output['carbon_savings']['ghg_savings_stock_per_year_per_ha'] = output['carbon_savings']['ghg_savings_stock_per_year_per_site']/output['site_data']['tot_area']
    output['carbon_savings']['ghg_savings_flow_per_year_per_site'] = output['rewet_outcomes']['activity_gwp_subtotal']*(-1)
    output['carbon_savings']['ghg_savings_flow_per_year_per_ha'] = output['carbon_savings']['ghg_savings_flow_per_year_per_site']/output['site_data']['tot_area']
    output['carbon_savings']['ghg_savings_product_use_per_year_per_site'] = crop_use.loc['ton_co2_per_site']['Values']*(-1)
    output['carbon_savings']['ghg_savings_product_use_per_year_per_ha'] = crop_use.loc['ton_co2_per_ha']['Values']*(-1)
    
    output['carbon_savings']['carbon_stock_peat_soil_start_year_tco2_per_site'] = c_content.loc['c_stock_tco2_per_ha']['Values']*output['site_data']['tot_area']
    output['carbon_savings']['carbon_stock_peat_soil_start_year_ton_c_per_site'] = c_content.loc['c_stock_ton_per_ha']['Values']*output['site_data']['tot_area']
    output['carbon_savings']['carbon_stock_peat_soil_start_year_tco2_per_ha'] = c_content.loc['c_stock_tco2_per_ha']['Values']
    output['carbon_savings']['carbon_stock_peat_soil_start_year_ton_c_per_ha'] = c_content.loc['c_stock_ton_per_ha']['Values']

    output['carbon_savings']['time_til_peat_is_lost_base_scenario'] = outcome.loc['base_scenario']['creditable_year']
    output['carbon_savings']['time_til_peat_is_lost_rewet_scenario)'] = outcome.loc['rewet_scenario']['creditable_year']
    
    with open('output_SET.json', 'w') as outfile:
        print(json.dumps(output, indent = 5), file = outfile)
    

In [93]:
Create_Output_tab('user_input_SET.json', sm_classes, data_tab, outcome, c_content_tab, crop_use_tab)